### SyN registration 시행 전 FOV cropping 필요 (DICOM에서)

In [1]:
import os
import pydicom
import SimpleITK as sitk
from tqdm.notebook import tqdm

In [2]:
# Define the directories for input and output
BASE_DIR = "E://data//single_TARE_dicomraw(pre,ap,pp)"
OUTPUT_DIR = "E://data//single_TARE_matchedwithFOVcropping(PP_fixed)_nifti"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [5]:
# Extract_z_range using pydicom
def extract_z_range(dicom_dir):
    z_positions = []
    for fname in os.listdir(dicom_dir):
        path = os.path.join(dicom_dir, fname)
        if not fname.lower().endswith(".dcm"):
            continue
        try:
            ds = pydicom.dcmread(path, stop_before_pixels=True)
            z = float(ds.ImagePositionPatient[2])  # z = slice location
            z_positions.append(z)
        except Exception as e:
            print(f"⚠️ Skipping {fname}: {e}")
            continue
    if not z_positions:
        raise ValueError(f"No valid DICOM slices found in {dicom_dir}")
    z_positions.sort()
    return z_positions[0], z_positions[-1]

# Crop SimpleITK image to match Z-range
def crop_to_z_range(image, z_min, z_max):
    origin = image.GetOrigin()
    spacing = image.GetSpacing()
    size = list(image.GetSize())
    z_positions = [origin[2] + i * spacing[2] for i in range(size[2])]

    z_indices = [
        i for i, z in enumerate(z_positions)
        if z_min <= z <= z_max or z_max <= z <= z_min
    ]
    if not z_indices:
        return None

    start_slice = min(z_indices)
    end_slice = max(z_indices) + 1
    size[2] = end_slice - start_slice
    index = [0, 0, start_slice]

    return sitk.RegionOfInterest(image, size, index)

# DICOM to SimpleITK volume
def load_dicom_series(dicom_dir):
    reader = sitk.ImageSeriesReader()
    series_ids = reader.GetGDCMSeriesIDs(dicom_dir)
    if not series_ids:
        raise ValueError(f"No DICOM series found in {dicom_dir}")
    series_files = reader.GetGDCMSeriesFileNames(dicom_dir, series_ids[0])
    reader.SetFileNames(series_files)
    return reader.Execute()

In [6]:
for patient_id in tqdm(os.listdir(BASE_DIR)):
    patient_dir = os.path.join(BASE_DIR, patient_id)
    if not os.path.isdir(patient_dir):
        continue

    try:
        pre_dir = os.path.join(patient_dir, "pre")
        ap_dir = os.path.join(patient_dir, "AP")
        pp_dir = os.path.join(patient_dir, "PP")
        if not all(os.path.isdir(p) for p in [pre_dir, ap_dir, pp_dir]):
            print(f"❌ {patient_id}: Missing DICOM folders")
            continue

        # 1. Extract Z-range from PP (fixed image)
        z_min, z_max = extract_z_range(pp_dir)

        # 2. Load DICOM volumes
        pre_img = load_dicom_series(pre_dir)
        ap_img = load_dicom_series(ap_dir)
        pp_img = load_dicom_series(pp_dir)  # no cropping needed

        # 3. Crop Pre and AP to match PP's Z-range
        pre_cropped = crop_to_z_range(pre_img, z_min, z_max)
        ap_cropped = crop_to_z_range(ap_img, z_min, z_max)

        if pre_cropped is None or ap_cropped is None:
            print(f"⚠️ {patient_id}: No overlapping Z-range")
            continue

        # 4. Save to output directory
        out_dir = os.path.join(OUTPUT_DIR, patient_id)
        os.makedirs(out_dir, exist_ok=True)

        sitk.WriteImage(pre_cropped, os.path.join(out_dir, "pre.nii.gz"))
        sitk.WriteImage(ap_cropped, os.path.join(out_dir, "AP.nii.gz"))
        sitk.WriteImage(pp_img, os.path.join(out_dir, "PP.nii.gz"))

    except Exception as e:
        print(f"❌ Error processing {patient_id}: {e}")

  0%|          | 0/144 [00:00<?, ?it/s]

Output directory structure:

```text
OUTPUT_DIR/
├── 00123/
│   ├── pre.nii.gz
│   ├── AP.nii.gz
│   └── PP.nii.gz
...
```